In [66]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

sum_tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
sum_model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

#summarizer = pipeline('summarization', model="sshleifer/distilbart-cnn-12-6")

def get_summary(text,max_size=150,min_size=50):
    text_length = len(text.split(" "))
    if (text_length > 1024):
        return "empty"
    if (text_length < 10):
        return text
    if (text_length < max_size):
        max_size = text_length - int(text_length/4)
    if (max_size < min_size):
        min_size = max_size
    if (min_size > max_size):
        max_size = min_size
    inputs = sum_tokenizer(text, return_tensors="pt", truncation=True, max_length=1024).input_ids
    outputs = sum_model.generate(inputs, max_new_tokens=100, do_sample=False)
    summary = sum_tokenizer.decode(outputs[0], skip_special_tokens=True)
    #summary = summarizer(text, max_length=max_size, min_length=min_size, do_sample=False)
    #text = summary[0]['summary_text'].strip()
    return summary

#print(get_summary("The purpose of this study was to demonstrate the use of SNA to understand public discourse on Twitter around the novel coronavirus . We focused on key public statements (\"communication events\") made by the WHO and its Director-General Dr. Tedros Ghebreyesus during the pandemic. As the leading international organization in the management of pandemics, we predicted that the accounts for the WHO (@WHO) and Dr. Ghebreyesus (@drtedros) would be leading the conversations about the coronavirus on Twitter. We expected the network to be highly centralized, with @WHO and @drtedros at the center of the conversation. Because networks are not static and public discourse on Twitter evolves, we expected the diameter and the modularity measures of the network to increase as more people tweeted about the issue and the discussion became more fragmented. Specifically, we had the following research questions: (1) What is the location of @WHO and @drtedros in the network of conversations around the coronavirus? Specifically, do they appear as central or core opinion leaders?; (2) Who are other opinion leaders on this topic on Twitter?; (3) How has the network changed across time, if at all?; (4) What inferences can we draw from the network properties regarding the successful transmission and adoption of public health messages on Twitter?",20))

In [59]:
import numpy as np
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def get_embedding(sentence):
    text = [sentence]
    embedding = sentence_model.encode(text)
    return embedding[0]

def get_cosine(u, v):
    score =  np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    return score

#print(get_embedding("Los autores sostienen que el protocolo debería proponer la realización de técnicas de detección del COVID-19 para el personal sanitario (test rápidos/ PCR) de manera frecuente y así establecer dos circuitos: uno para el personal sanitario (limpio) y otro (sucio) para el resto de las patologías. El personal médico formado debería estar en conocimiento que debido a las múltiples manifestaciones clínicas del virus (conjuntivitis, diarrea, vómitos, fiebre, perniosis, urticarias, cefaleas, anosmia, disnea, tos, dolor abdominal, etc.), y especialmente teniendo en cuenta a los pacientes asintomáticos, se debería considerar a todo paciente como posible caso de transmisión de COVID-19 con lo cual es recomendable el uso de EPI ante todo paciente, independientemente de la clínica que presente. Es de destacar, que como bien indica el protocolo del Ministerio, en las medidas de prevención de contagio para los profesionales sanitarios cuando acuden al domicilio de un paciente la vivienda debería ser considerada como una zona contaminada. Advertimos que esta lógica no se traslada a los centros de salud, cuyos circuitos por donde circulan los pacientes asintomáticos y sospechosos de COVID-19 deberían ser también considerados zonas contaminadas. En la actualidad los pacientes asintomáticos deberían considerarse un reto en su precoz detección mediante técnica RT-PCR para controlar los contactos estrechos y disminuir la propagación de la pandemia. Es destacable un estudio que analiza las glándulas salivares como posible reservorio del virus en asintomáticos. De esta forma, el RNA del SARCOV podría ser detectado en saliva antes que en secreciones respiratorias 5 . Agregar que una publicación reciente analizó múltiples muestras de heces y concluyó que es posible detectar el virus por RT-PCR en pacientes asintomáticos hasta 17 días después de la última exposición al virus 6 . La estimación del porcentaje de asintomáticos mejorará la comprensión de la transmisión de COVID-19 así como ampliará sustancialmente el conocimiento de la enfermedad que causa, proporcionando una idea de la propagación de la epidemia 7 . En este sentido, mejorar los protocolos de actuación en relación con los pacientes asintomáticos ayudará a disminuir el número de incidencia del virus. La atención primaria es un pilar fundamental para la gestión de la pandemia, que debe ser reforzada con el incremento de personal sanitario, dotación de EPI, test diagnósticos e infraestructura adecuada por parte de las CC.AA. Asimismo, es necesario realizar diagnósticos precoces de casos asintomáticos, presintomáticos y aislamiento de casos en contacto estrecho. Para finalizar, en concordancia con Gandhi y colaboradores 8 , afirmamos que los pacientes asintomáticos representan el talón de Aquiles de la transmisión de la enfermedad, que debe ser abordado, especialmente en los protocolos de actuación desde la atención primaria. Bibliografía 1. WHO. WHO Director-General's opening remarks at the media briefing on COVID-19 -11 March 2020 [consultado 13 Jun 2020]. Disponible en: https://www.who.int/dg/speeches/detail/who-director-general-s-openingremarks-at-the-media-briefing-on-covid-19---11-march-2020 2. Rodríguez Labrada R, Vázquez Mojena Y., Velázquez Pérez, L."))

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
ent_tokenizer = AutoTokenizer.from_pretrained("lighteternal/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext-finetuned-mnli")
ent_model = AutoModelForSequenceClassification.from_pretrained("lighteternal/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext-finetuned-mnli")

def get_entailment(premise, hypothesis):
    x = ent_tokenizer.encode(premise, hypothesis, return_tensors='pt',truncation_strategy='only_first')
    logits = ent_model(x)[0]
    probs = logits.softmax(dim=1)
    values = np.around(probs.cpu().detach().numpy(),3)
    result = {
        'entailment':values[0][0],
        'neutral':values[0][1],
        'contradiction':values[0][2],
    }
    return result
    
premise = 'Tips from an Italian doctor on the Coronavirus'
hypothesis = 'Behavioral tips for the prevention of novel coronavirus infection by medical institutes -January 29, 2020, KCDC [4] 1. Wear a mask or other protective device when caring for patients with respiratory diseases 2. Check the international travel history of patients, including countries where there is a coronavirus epidemic. 3. If a novel coronavirus infection is suspected, report the patient to the health center of the local jurisdiction.'
print(get_entailment(premise,hypothesis))
    

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'entailment': 0.001, 'neutral': 0.001, 'contradiction': 0.998}


In [11]:
premise = 'Risk factors for hospitalization due to COVID-19 (no risk factors for hospitalization due to COVID 19 versus ≥1 risk factor for hospitalization due to COVID-19)'
hypothesis = 'Older adults and people with severe underlying chronic conditions, such as heart disease, lung disease, or diabetes, seem to be at increased risk for more severe complications from COVID-19. Talk to your health care provider about additional measures you might take to protect yourself.'
print(get_entailment(hypothesis,premise))


{'entailment': 0.131, 'neutral': 0.109, 'contradiction': 0.76}


In [76]:
import json 
import time
from random import randint
from time import sleep

#name = "covid_5w1h_dataset_170_news_english_evidences"
#name = "eswa_english_evidences"
name = "covid_5w1h_english_evidences"
with open("../evidences/"+name+".json", "r") as read_file:
    data = json.load(read_file)
    news_size = len(data)
    counter = 1
    try: 
        total_tp = 0
        total_tn = 0
        total_fp = 0
        total_fn = 0
        for i in data:
            status = "FAKE"
            if (i["value_true"] == "True"):
                status = "TRUE"
            print(counter,"/",news_size,"[",status,"]",i['id'],":", i["value_reliability"])            
            counter += 1
            tp = 0
            tn = 0
            fp = 0
            fn = 0
            for a in i['annotation_values']:
                ref_text = a["text_en"]
                if (len(ref_text.split(" ")) > 1024):
                    ref_text = get_summary(a["text_en"],1024)
                ref_vector = get_embedding(a["text_en"])
                ref_size = len(a["text_en"].split(" "))
                labels = []
                total = 0
                positive = 0
                for w in a['5w1h_value']:
                    a_label =  w['5w1h_label']
                    total += 1
                    reliable = "-"
                    if (w['5w1h_reliability'] == 'Reliable'):
                        reliable = "+"
                        positive +=1
                    labels.append(a_label+"("+reliable+")")
                is_reliable = (positive > (total-positive))
                distances = []
                new_evidences = []
                for e in a['evidences']:
                    e_sum = get_summary(e,ref_size)
                    e_vector = get_embedding(e_sum)
                    e_distance = get_cosine(ref_vector,e_vector)
                    distance = round(e_distance, 2)
                    if (distance >= 0.50):
                        distances.append(e_distance)
                        new_evidences.append({'value':e,'score':float(distance)})
                if (is_reliable):
                    if (len(distances)>0):
                        tp += 1
                        total_tp += 1
                    else:
                        fn += 1
                        total_fn += 1
                else:
                    if (len(distances)>0):
                        fp += 1
                        total_fp += 1
                    else:
                        tn += 1
                        total_tn += 1
                print("\t","q:",len(a['questions']),"e:",len(a['evidences']),labels,distances)
                del a['questions']
                if (len(new_evidences)>0):
                    a['evidences']=new_evidences
                else:
                    del a['evidences']
            micro_precision = 1.0
            if ((tp+fp)>0.0):
                micro_precision = tp / (tp+fp)
            micro_recall = 1.0
            if ((tp+fn)>0.0):
                micro_recall = tp / (tp+fn)
            micro_average = 1.0
            if ((tp+fn+tn+fp)>0.0):
                micro_average = (tp + tn) / (tp + tn + fp + fn)
            micro_f = 1.0
            if ((micro_precision+micro_recall)>0.0):
                micro_f = 2 * ((micro_precision*micro_recall)/(micro_precision+micro_recall))
            print("\t## micro_p:",round(micro_precision,2),"micro_r:",round(micro_recall,2),"micro_avg:",round(micro_average,2),"micro_f:",round(micro_f,2))
            print("\t-------------------")
        macro_precision = 1.0
        if ((total_tp+total_fp)>0.0):
            macro_precision = total_tp / (total_tp+total_fp)
        macro_recall = 1.0
        if ((total_tp+total_fn)>0.0):
            macro_recall = total_tp / (total_tp+total_fn)
        macro_average = (total_tp + total_tn) / (total_tp + total_tn + total_fp + total_fn)
        macro_f = 2 * ((macro_precision*macro_recall)/(macro_precision+macro_recall))
        print("\t-------------------")
        print("## macro_p:",round(macro_precision,2),"macro_r:",round(macro_recall,2),"macro_avg:",round(macro_average,2),"macro_f:",round(macro_f,2))
    except Exception as e:
        print("Error on execution: " + str(e))
        
    # Closing file
    input_file.close()
    
    output_file = open("../evidences/"+name+"_strict.json", "w")
    output_file.write(json.dumps(data, ensure_ascii=False, indent=4))


1 / 55 [ FAKE ] Health_225 : NC
	 q: 0 e: 0 ['Who(+)', 'What(-)'] []
	 q: 2 e: 3 ['Who(-)', 'What(-)', 'How(-)'] []
	 q: 2 e: 1 ['What(-)', 'Who(+)', 'How(-)'] []
	 q: 2 e: 0 ['When(-)', 'What(-)', 'Who(-)'] []
	 q: 6 e: 0 ['Who(-)', 'What(-)', 'Where(-)', 'When(-)', 'What(-)', 'Why(-)'] []
	 q: 1 e: 2 ['What(-)', 'Where(+)'] []
	 q: 0 e: 0 ['Who(-)', 'What(-)'] []
	 q: 3 e: 1 ['What(-)', 'Where(+)', 'Why(-)', 'Who(-)'] []
	 q: 1 e: 1 ['What(-)'] []
	 q: 1 e: 1 ['What(-)'] []
	 q: 3 e: 6 ['What(-)', 'Where(+)'] [0.5048048, 0.5290614, 0.5906574]
	## micro_p: 0.0 micro_r: 1.0 micro_avg: 0.91 micro_f: 0.0
	-------------------
2 / 55 [ TRUE ] Health_231 : C
	 q: 0 e: 0 ['What(-)', 'Why(-)'] []
	 q: 1 e: 2 ['What(+)', 'Where(+)', 'How(+)'] []
	 q: 0 e: 0 ['Who(+)', 'Where(+)', 'When(+)', 'Who(+)', 'What(+)', 'Where(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'How(+)'] []
	 q: 1 e: 0 ['What(+)', 'Who(+)', 'Where(+)'] []
	 q: 2 e: 4 ['What(+)'] [0.5454356, 0.7198651, 0.544278, 0.824875]
	 q: 2 

	 q: 3 e: 2 ['Where(+)', 'Who(+)', 'Why(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'Why(+)', 'Why(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'Where(+)', 'Why(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'Who(+)', 'When(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'Where(+)'] []
	## micro_p: 1.0 micro_r: 0.09 micro_avg: 0.09 micro_f: 0.17
	-------------------
14 / 55 [ TRUE ] Health_220 : C
	 q: 2 e: 3 ['How(+)', 'What(+)'] []
	 q: 1 e: 1 ['When(+)', 'What(+)', 'Where(+)', 'Why(+)', 'Where(+)'] []
	 q: 1 e: 0 ['What(+)', 'When(+)', 'Where(+)', 'Who(+)', 'What(+)', 'Why(+)'] []
	 q: 3 e: 1 ['Who(+)', 'When(+)', 'How(+)', 'What(+)', 'Where(+)'] []
	 q: 3 e: 6 ['What(+)', 'How(+)'] []
	 q: 0 e: 0 ['What(+)', 'How(+)'] []
	 q: 1 e: 1 ['What(+)', 'How(+)'] []
	 q: 0 e: 0 ['What(+)'] []
	 q: 2 e: 4 ['What(+)'] []
	 q: 1 e: 2 ['What(+)', 'How(+)'] []
	 q: 1 e: 2 ['What(+)', 'Who(+)', 'Who(+)'] []
	## micro_p: 1.0 micro_r: 0.0 micro_avg: 0.0 micro_f: 0.0
	-------------------
15 / 55 [ FAKE ] Health_221

	 q: 5 e: 4 ['Who(+)', 'What(+)', 'Where(+)', 'When(+)', 'Who(+)', 'Where(+)', 'What(-)', 'When(+)'] []
	 q: 3 e: 0 ['When(+)', 'Who(+)', 'What(-)', 'How(-)'] []
	 q: 2 e: 0 ['When(+)', 'What(-)', 'How(+)', 'Who(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(-)', 'How(+)'] []
	## micro_p: 1.0 micro_r: 0.2 micro_avg: 0.64 micro_f: 0.33
	-------------------
26 / 55 [ FAKE ] Covid-19_45 : NC
	 q: 0 e: 0 ['What(-)', 'Who(-)'] []
	 q: 0 e: 0 ['What(-)', 'Who(-)', 'Where(+)', 'Who(-)'] []
	 q: 1 e: 0 ['What(-)'] []
	 q: 1 e: 2 ['What(-)', 'Who(-)', 'How(-)'] []
	 q: 1 e: 2 ['What(-)', 'How(-)', 'What(-)', 'Who(+)', 'How(-)'] [0.520971]
	 q: 2 e: 2 ['Where(+)', 'What(-)'] []
	 q: 5 e: 5 ['Who(-)', 'What(-)', 'Where(+)', 'Why(-)'] []
	 q: 2 e: 3 ['What(-)'] []
	 q: 2 e: 3 ['Who(-)', 'What(-)', 'Who(-)'] []
	 q: 3 e: 0 ['What(-)', 'Who(+)', 'When(-)', 'Who(-)'] []
	 q: 0 e: 0 ['Who(-)', 'What(-)', 'Who(-)', 'What(-)'] []
	## micro_p: 0.0 micro_r: 1.0 micro_avg: 0.91 micro_f: 0.0
	-------------------
27 /

	 q: 5 e: 5 ['How(-)', 'Who(+)', 'What(-)', 'Who(+)', 'Who(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(-)', 'Why(-)'] []
	 q: 2 e: 0 ['Why(+)', 'What(+)', 'Who(-)', 'Who(+)'] []
	 q: 3 e: 0 ['Who(-)', 'What(+)', 'When(+)', 'Who(+)', 'Who(-)', 'What(-)'] []
	## micro_p: 1.0 micro_r: 0.2 micro_avg: 0.6 micro_f: 0.33
	-------------------
39 / 55 [ FAKE ] Covid-19_158 : NC
	 q: 2 e: 1 ['Who(-)', 'What(-)', 'Where(+)', 'Why(-)', 'Where(+)'] [0.5784498]
	 q: 0 e: 0 ['What(-)', 'Why(-)'] []
	 q: 4 e: 0 ['What(-)', 'When(+)', 'Who(-)', 'Why(-)', 'Who(-)'] []
	 q: 0 e: 0 ['What(-)'] []
	 q: 2 e: 0 ['What(+)', 'Who(+)'] []
	 q: 0 e: 0 ['What(-)'] []
	 q: 0 e: 0 ['What(+)', 'How(+)', 'Where(+)', 'When(+)'] []
	 q: 2 e: 4 ['What(-)', 'Where(+)', 'How(-)'] []
	 q: 2 e: 0 ['What(-)', 'Where(+)', 'Who(+)', 'Why(-)'] []
	 q: 0 e: 0 ['What(-)'] []
	 q: 0 e: 0 ['What(-)', 'Where(+)'] []
	## micro_p: 0.0 micro_r: 0.0 micro_avg: 0.73 micro_f: 1.0
	-------------------
40 / 55 [ FAKE ] Health_205 : NC
	 q: 3 e: 3 

	 q: 2 e: 1 ['What(-)', 'Where(+)'] []
	## micro_p: 0.0 micro_r: 0.0 micro_avg: 0.75 micro_f: 1.0
	-------------------
52 / 55 [ TRUE ] Covid-19_34 : C
	 q: 1 e: 0 ['What(+)', 'Who(+)'] []
	 q: 1 e: 0 ['When(+)', 'Who(+)', 'What(+)', 'How(+)'] []
	 q: 2 e: 0 ['Who(+)', 'What(+)', 'Why(+)'] []
	 q: 1 e: 0 ['What(+)', 'When(+)', 'Where(+)'] []
	 q: 1 e: 2 ['What(-)'] []
	 q: 0 e: 0 ['What(-)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'Where(+)'] []
	 q: 5 e: 0 ['When(+)', 'Who(+)', 'What(+)', 'Where(+)', 'Why(+)'] []
	 q: 2 e: 0 ['Who(+)', 'What(+)'] []
	 q: 0 e: 0 ['What(+)', 'When(+)', 'Where(+)'] []
	 q: 3 e: 0 ['What(+)', 'When(+)', 'Where(+)', 'Where(+)', 'Who(+)'] []
	## micro_p: 1.0 micro_r: 0.0 micro_avg: 0.18 micro_f: 0.0
	-------------------
53 / 55 [ TRUE ] Health_214 : C
	 q: 4 e: 0 ['What(-)', 'Who(+)', 'Where(+)', 'Why(+)', 'How(-)'] []
	 q: 2 e: 0 ['What(+)', 'Who(+)', 'Who(+)'] []
	 q: 2 e: 1 ['Where(+)', 'Who(+)', 'What(+)', 'Where(+)'] []
	 q: 2 e: 4 ['Who(+)', 'What(+)', '

In [78]:
import json 
import time
from random import randint
from time import sleep

name = "covid_5w1h_dataset_170_news_english_evidences"
with open("../evidences/"+name+".json", "r") as read_file:
    data = json.load(read_file)
    news_size = len(data)
    counter = 1
    try: 
        total_tp = 0
        total_tn = 0
        total_fp = 0
        total_fn = 0
        for i in data:
            status = "FAKE"
            if (i["value_reliability"] == "1"):
                status = "TRUE"
            print(counter,"/",news_size,"[",status,"]",i['id'],":", i["value_reliability"])            
            counter += 1
            tp = 0
            tn = 0
            fp = 0
            fn = 0
            for a in i['annotation_values']:
                ref_text = a["text_en"]
                if (len(ref_text.split(" ")) > 1024):
                    ref_text = get_summary(a["text_en"],1024)
                ref_vector = get_embedding(a["text_en"])
                ref_size = len(a["text_en"].split(" "))
                labels = []
                total = 0
                positive = 0
                for w in a['5w1h_value']:
                    a_label =  w['5w1h_label']
                    total += 1
                    reliable = "-"
                    if (w['5w1h_reliability'] == 'Reliable'):
                        reliable = "+"
                        positive +=1
                    labels.append(a_label+"("+reliable+")")
                is_reliable = (positive > (total-positive))
                distances = []
                new_evidences = []
                for e in a['evidences']:
                    e_sum = get_summary(e,ref_size)
                    e_vector = get_embedding(e_sum)
                    e_distance = get_cosine(ref_vector,e_vector)
                    distance = round(e_distance, 2)
                    if (distance >= 0.50):
                        distances.append(e_distance)
                        new_evidences.append({'value':e,'score':float(distance)})
                if (is_reliable):
                    if (len(distances)>0):
                        tp += 1
                        total_tp += 1
                    else:
                        fn += 1
                        total_fn += 1
                else:
                    if (len(distances)>0):
                        fp += 1
                        total_fp += 1
                    else:
                        tn += 1
                        total_tn += 1
                print("\t","q:",len(a['questions']),"e:",len(a['evidences']),labels,distances)
                del a['questions']
                if (len(new_evidences)>0):
                    a['evidences']=new_evidences
                else:
                    del a['evidences']
            micro_precision = 1.0
            if ((tp+fp)>0.0):
                micro_precision = tp / (tp+fp)
            micro_recall = 1.0
            if ((tp+fn)>0.0):
                micro_recall = tp / (tp+fn)
            micro_average = 1.0
            if ((tp+fn+tn+fp)>0.0):
                micro_average = (tp + tn) / (tp + tn + fp + fn)
            micro_f = 1.0
            if ((micro_precision+micro_recall)>0.0):
                micro_f = 2 * ((micro_precision*micro_recall)/(micro_precision+micro_recall))
            print("\t## micro_p:",round(micro_precision,2),"micro_r:",round(micro_recall,2),"micro_avg:",round(micro_average,2),"micro_f:",round(micro_f,2))
            print("\t-------------------")
        macro_precision = 1.0
        if ((total_tp+total_fp)>0.0):
            macro_precision = total_tp / (total_tp+total_fp)
        macro_recall = 1.0
        if ((total_tp+total_fn)>0.0):
            macro_recall = total_tp / (total_tp+total_fn)
        macro_average = (total_tp + total_tn) / (total_tp + total_tn + total_fp + total_fn)
        macro_f = 2 * ((macro_precision*macro_recall)/(macro_precision+macro_recall))
        print("\t-------------------")
        print("## macro_p:",round(macro_precision,2),"macro_r:",round(macro_recall,2),"macro_avg:",round(macro_average,2),"macro_f:",round(macro_f,2))
    except Exception as e:
        print("Error on execution: " + str(e))
        
    # Closing file
    input_file.close()
    
    output_file = open("../evidences/"+name+"_strict.json", "w")
    output_file.write(json.dumps(data, ensure_ascii=False, indent=4))

1 / 116 [ FAKE ] 10-Morelosmagazzine : 0
	 q: 2 e: 3 ['Who(+)', 'What(-)', 'Who(+)', 'What(-)'] [0.6307281]
	 q: 1 e: 0 ['Where(+)', 'What(-)', 'Who(-)', 'Who(+)'] []
	 q: 2 e: 3 ['What(-)'] []
	 q: 4 e: 3 ['When(-)', 'Who(+)', 'What(-)', 'Who(+)', 'Where(+)', 'What(-)', 'Who(+)', 'Where(+)', 'What(-)', 'Who(+)', 'What(-)', 'Who(-)'] []
	 q: 0 e: 0 ['What(-)', 'Who(+)', 'What(-)', 'Who(+)', 'When(-)'] []
	 q: 2 e: 3 ['Who(+)', 'What(-)', 'Where(+)'] [0.5273579]
	## micro_p: 0.5 micro_r: 1.0 micro_avg: 0.83 micro_f: 0.67
	-------------------
2 / 116 [ FAKE ] NF_NF_1841_mpr21_info : 0
	 q: 2 e: 1 ['What(+)', 'Where(+)'] [0.6245288]
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'When(+)', 'What(-)'] []
	 q: 1 e: 1 ['What(-)', 'Where(+)', 'Where(+)', 'How(+)', 'Who(+)'] [0.6323591]
	 q: 2 e: 2 ['Who(-)', 'Where(+)', 'What(-)', 'Where(+)', 'Why(+)'] [0.5335184]
	 q: 0 e: 0 ['What(-)'] []
	 q: 1 e: 2 ['Why(-)'] []
	 q: 5 e: 3 ['What(+)', 'Who(+)', 'Where(+)', 'Where(+)'] []
	 q: 0 e: 0 ['What(+)', 'Wher

	 q: 1 e: 2 ['Where(+)', 'What(+)', 'Where(+)', 'Why(+)'] []
	 q: 3 e: 4 ['What(+)', 'Why(+)', 'What(+)', 'Where(+)', 'Why(+)'] []
	 q: 0 e: 0 ['What(+)', 'What(+)', 'Why(+)', 'Who(+)', 'Where(+)'] []
	## micro_p: 1.0 micro_r: 0.11 micro_avg: 0.11 micro_f: 0.2
	-------------------
12 / 116 [ TRUE ] F_04-11-2021_20-48-39_www_20minutos_es : 1
	 q: 2 e: 2 ['What(+)', 'Where(+)', 'Where(+)'] []
	 q: 2 e: 2 ['What(+)'] []
	 q: 2 e: 4 ['Who(+)', 'What(+)', 'Who(+)'] []
	 q: 0 e: 0 ['What(+)', 'Where(+)', 'Why(+)'] []
	 q: 0 e: 0 ['What(+)', 'When(+)', 'Who(+)'] []
	 q: 2 e: 4 ['What(+)'] [0.5234032, 0.59112084]
	## micro_p: 1.0 micro_r: 0.17 micro_avg: 0.17 micro_f: 0.29
	-------------------
13 / 116 [ FAKE ] NF_04-11-2021_20-35-55_www_eldiestro_es : 0
	 q: 0 e: 0 ['Who(+)', 'What(-)', 'Where(+)'] []
	 q: 1 e: 2 ['When(-)', 'Where(+)', 'What(-)'] []
	 q: 0 e: 0 ['Where(+)', 'Who(+)', 'What(+)'] []
	 q: 1 e: 2 ['When(+)', 'Where(+)', 'What(+)'] [0.6072352]
	 q: 2 e: 0 ['What(-)', 'How(-)', 'W

	 q: 2 e: 4 ['Who(+)', 'What(+)', 'Why(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'Where(+)'] []
	 q: 1 e: 0 ['Who(+)'] []
	 q: 0 e: 0 ['What(+)', 'Why(+)'] []
	 q: 0 e: 0 ['Who(+)'] []
	 q: 1 e: 2 ['What(+)'] []
	 q: 3 e: 1 ['Who(+)', 'What(+)', 'When(+)', 'Who(+)', 'What(+)'] [0.6069816]
	 q: 2 e: 0 ['Who(+)', 'What(+)', 'Who(+)'] []
	 q: 0 e: 0 ['What(+)', 'When(+)'] []
	## micro_p: 1.0 micro_r: 0.06 micro_avg: 0.06 micro_f: 0.12
	-------------------
22 / 116 [ TRUE ] NF_04-11-2021_21-50-02_www_eje21_com_co : 1
	 q: 2 e: 0 ['Who(+)', 'What(+)', 'How(+)'] []
	 q: 4 e: 0 ['Who(+)', 'When(+)', 'What(+)', 'How(+)'] []
	 q: 2 e: 0 ['Who(+)', 'What(+)', 'How(+)', 'Who(+)'] []
	 q: 1 e: 0 ['When(+)', 'Who(+)', 'Who(+)', 'Who(+)', 'What(+)', 'Why(+)'] []
	 q: 2 e: 0 ['Who(+)', 'What(+)', 'When(+)'] []
	 q: 2 e: 0 ['Who(+)', 'What(+)'] []
	 q: 2 e: 3 ['Who(+)', 'What(+)'] [0.51762843]
	 q: 2 e: 0 ['When(+)', 'Who(+)', 'Where(+)', 'What(+)', 'How(+)'] []
	 q: 3 e: 0 ['When(+)', 'Who(+)', 'What

	 q: 3 e: 1 ['What(-)', 'Where(+)', 'How(+)'] [0.60582775]
	 q: 2 e: 0 ['When(-)', 'What(-)', 'Who(+)', 'Who(-)', 'Where(+)'] []
	 q: 2 e: 2 ['What(-)', 'Where(-)', 'Why(-)'] []
	 q: 1 e: 2 ['Where(+)'] []
	 q: 1 e: 2 ['Who(+)', 'What(-)', 'How(+)'] []
	 q: 2 e: 2 ['Where(+)', 'What(+)'] [0.6507058, 0.5080676]
	 q: 3 e: 0 ['Who(+)', 'Where(+)', 'When(+)', 'What(+)'] []
	 q: 3 e: 0 ['Who(+)', 'What(+)', 'When(+)'] []
	 q: 2 e: 0 ['Who(+)', 'When(+)', 'Who(+)', 'What(+)', 'Where(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'When(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)'] []
	 q: 2 e: 0 ['When(+)', 'Who(+)', 'What(+)'] []
	 q: 3 e: 5 ['Who(+)', 'What(+)'] [0.50008553]
	 q: 0 e: 0 ['Where(+)', 'What(-)'] []
	 q: 1 e: 1 ['Where(+)', 'When(+)', 'Who(+)', 'What(-)', 'Why(-)', 'How(-)'] [0.6187658]
	## micro_p: 0.75 micro_r: 0.27 micro_avg: 0.4 micro_f: 0.4
	-------------------
32 / 116 [ TRUE ] F_04-11-2021_21-50-53_elpais_com : 1
	 q: 3 e: 0 ['Who(+)', 'What(+)', 'Who(+)', 'When(+)'] []
	 q: 2 e

	 q: 3 e: 1 ['Who(-)', 'What(-)', 'Why(-)', 'Who(+)'] []
	 q: 2 e: 0 ['Who(+)', 'Where(+)', 'What(-)', 'Who(-)', 'Where(-)'] []
	 q: 2 e: 0 ['Who(+)', 'What(+)', 'When(-)', 'Where(+)', 'What(-)', 'When(+)'] []
	 q: 1 e: 0 ['Where(+)', 'When(-)', 'What(+)'] []
	 q: 3 e: 4 ['Who(-)', 'What(-)', 'Where(+)'] []
	 q: 1 e: 0 ['What(-)', 'Who(-)', 'Who(+)', 'What(-)'] []
	 q: 2 e: 4 ['Who(-)', 'What(-)'] []
	 q: 2 e: 0 ['Who(-)', 'What(-)'] []
	## micro_p: 1.0 micro_r: 0.0 micro_avg: 0.75 micro_f: 0.0
	-------------------
40 / 116 [ TRUE ] F_04-11-2021_20-34-29_www_20minutos_es : 1
	 q: 2 e: 2 ['What(+)', 'Where(+)', 'When(+)'] [0.60307246]
	 q: 2 e: 3 ['What(+)', 'Who(+)'] []
	 q: 1 e: 1 ['What(+)', 'When(+)', 'Where(+)'] [0.6000453]
	 q: 2 e: 2 ['When(+)', 'Who(+)', 'What(+)', 'Where(+)', 'What(+)'] [0.6638623, 0.81974375]
	 q: 2 e: 4 ['What(+)', 'Who(+)'] []
	 q: 2 e: 0 ['Who(+)', 'How(+)', 'What(+)', 'Where(+)'] []
	 q: 1 e: 0 ['Who(+)', 'Where(+)', 'What(+)'] []
	 q: 1 e: 0 ['What(+)', '

...

	 q: 1 e: 1 ['Who(+)', 'How(-)', 'What(-)'] []
	 q: 1 e: 1 ['What(-)'] []
	## micro_p: 0.0 micro_r: 0.0 micro_avg: 0.71 micro_f: 1.0
	-------------------
72 / 116 [ FAKE ] NF_04-11-2021_22-13-36_www_periodistadigital_com : 0
	 q: 2 e: 3 ['Who(+)', 'Where(+)', 'What(+)'] [0.5047019]
	 q: 3 e: 0 ['What(-)', 'Who(+)', 'When(+)'] []
	 q: 2 e: 0 ['Who(+)', 'What(-)'] []
	 q: 0 e: 0 ['When(-)', 'Who(-)', 'What(+)', 'Where(+)'] []
	 q: 2 e: 4 ['Who(-)', 'What(+)', 'How(+)', 'Where(-)'] [0.5827066]
	 q: 0 e: 0 ['Who(+)', 'Where(+)', 'What(-)', 'How(-)'] []
	 q: 0 e: 0 ['Who(+)', 'Where(+)', 'When(+)', 'What(+)', 'Where(+)', 'Where(+)'] []
	 q: 3 e: 0 ['What(-)', 'Who(+)', 'When(+)'] []
	 q: 0 e: 0 ['Where(+)', 'What(+)', 'Who(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(-)', 'Why(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)'] []
	 q: 2 e: 2 ['Where(+)', 'What(+)', 'Who(+)'] [0.59944654, 0.52298206]
	 q: 1 e: 2 ['What(+)'] []
	 q: 3 e: 0 ['Where(+)', 'What(+)', 'Who(+)', 'Why(-)', 'Who(-)'] []
	 q: 1 e: 0 [

	 q: 1 e: 1 ['Who(-)', 'What(-)', 'Where(-)'] [0.54030734]
	 q: 2 e: 3 ['Who(-)', 'What(+)', 'Why(-)'] []
	 q: 2 e: 1 ['Who(+)', 'What(+)', 'Who(+)'] [0.5161882]
	 q: 0 e: 0 ['How(+)', 'Where(+)', 'Who(+)', 'How(+)', 'What(+)'] []
	 q: 2 e: 2 ['What(+)', 'How(+)'] []
	 q: 1 e: 2 ['What(+)', 'Who(-)', 'How(+)'] []
	 q: 1 e: 0 ['When(+)', 'Who(+)', 'What(+)', 'When(+)', 'Why(+)', 'Who(+)'] []
	 q: 0 e: 0 ['Who(+)', 'Where(+)', 'What(+)'] []
	 q: 0 e: 0 ['Who(+)', 'Who(+)', 'What(+)'] []
	 q: 1 e: 0 ['Who(+)', 'How(+)', 'What(+)'] []
	## micro_p: 0.5 micro_r: 0.08 micro_avg: 0.24 micro_f: 0.13
	-------------------
82 / 116 [ TRUE ] F_04-11-2021_21-38-12_www_20minutos_es : 1
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'Who(+)', 'When(+)'] []
	 q: 2 e: 4 ['Who(+)', 'What(+)'] [0.55136156]
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'When(+)', 'When(+)', 'Where(+)'] []
	 q: 0 e: 0 ['What(+)', 'Who(+)', 'When(+)', 'Where(+)'] []
	 q: 1 e: 1 ['What(+)', 'Where(+)'] [0.56609654]
	 q: 1 e: 0 ['Who(+)', 'When(+)', '

	 q: 3 e: 4 ['Who(-)', 'Where(+)', 'What(+)', 'Who(+)', 'What(-)', 'Why(+)'] []
	 q: 2 e: 4 ['What(-)', 'Where(+)', 'Who(-)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)'] []
	 q: 0 e: 0 ['When(-)', 'What(-)', 'How(+)'] []
	 q: 1 e: 0 ['Who(+)', 'When(+)', 'What(-)'] []
	 q: 1 e: 0 ['Who(-)', 'When(-)', 'What(-)'] []
	 q: 0 e: 0 ['Who(-)', 'Where(+)', 'What(-)', 'Where(+)'] []
	 q: 2 e: 0 ['When(+)', 'Who(+)', 'What(+)', 'Why(-)'] []
	 q: 1 e: 0 ['Who(-)', 'What(-)'] []
	 q: 1 e: 1 ['What(-)', 'Who(-)'] []
	 q: 0 e: 0 ['What(-)', 'Who(-)'] []
	## micro_p: 1.0 micro_r: 0.0 micro_avg: 0.64 micro_f: 0.0
	-------------------
95 / 116 [ FAKE ] NF_04-11-2021_21-14-02_www_periodistadigital_com : 0
	 q: 2 e: 2 ['What(-)', 'Who(+)', 'Who(+)'] []
	 q: 1 e: 2 ['What(+)', 'Who(+)', 'What(+)', 'Where(+)', 'What(+)', 'Where(+)'] [0.6969714, 0.574484]
	 q: 0 e: 0 ['Who(-)', 'What(-)', 'Why(-)', 'Where(+)'] []
	 q: 0 e: 0 ['What(+)', 'Where(+)'] []
	 q: 1 e: 2 ['What(+)'] [0.6969714, 0.574484]
	 q: 2 e: 0 ['Wh

	 q: 6 e: 9 ['Who(+)', 'What(-)', 'Who(+)', 'Who(+)', 'What(-)'] [0.61198634, 0.6739334, 0.6150892, 0.6790831, 0.6563813, 0.5667118, 0.625764]
	 q: 1 e: 2 ['Who(+)', 'What(-)', 'Who(+)', 'Who(+)', 'What(-)', 'Who(+)'] [0.59225833]
	 q: 4 e: 1 ['What(+)', 'Who(+)', 'How(-)', 'When(+)'] []
	 q: 2 e: 0 ['What(-)', 'Who(+)', 'What(-)', 'Who(-)'] []
	 q: 1 e: 2 ['Why(+)'] []
	 q: 3 e: 4 ['When(+)', 'Who(+)', 'What(+)'] [0.5938705]
	 q: 4 e: 0 ['What(+)', 'Where(+)', 'Who(-)', 'Who(+)', 'What(-)'] []
	 q: 0 e: 0 ['How(-)'] []
	 q: 0 e: 0 ['Who(-)', 'What(-)', 'Who(+)', 'What(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'Why(+)', 'Where(+)'] []
	 q: 2 e: 3 ['Who(+)', 'What(+)', 'How(+)', 'Where(+)'] [0.53584456]
	 q: 3 e: 1 ['What(-)', 'When(+)', 'Who(-)'] []
	 q: 0 e: 0 ['What(+)', 'How(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'How(+)', 'Who(+)'] []
	 q: 1 e: 2 ['What(-)'] [0.58831364]
	 q: 1 e: 2 ['What(-)'] []
	 q: 3 e: 5 ['When(+)', 'Who(+)', 'What(+)'] [0.5056442, 0.5762854, 0.5231054]
	 q

	 q: 6 e: 4 ['Who(+)', 'Who(+)', 'What(+)'] []
	 q: 3 e: 0 ['What(+)', 'When(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'Why(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'Where(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)'] []
	 q: 1 e: 0 ['What(+)', 'Where(+)'] []
	 q: 1 e: 0 ['Who(+)', 'What(+)', 'How(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)'] []
	 q: 2 e: 3 ['Who(+)', 'What(+)', 'How(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'How(+)'] []
	## micro_p: 1.0 micro_r: 0.0 micro_avg: 0.0 micro_f: 0.0
	-------------------
114 / 116 [ FAKE ] NF_04-11-2021_20-29-17_www_eldiestro_es : 0
	 q: 1 e: 2 ['How(-)', 'What(-)', 'Who(-)', 'Where(+)', 'Who(+)'] []
	 q: 1 e: 0 ['What(-)', 'How(-)', 'Who(+)'] []
	 q: 1 e: 0 ['What(+)', 'Where(+)', 'Why(-)'] []
	 q: 4 e: 0 ['What(-)', 'Who(+)', 'What(+)', 'Where(+)'] []
	 q: 2 e: 1 ['What(-)', 'How(-)'] []
	 q: 2 e: 0 ['Who(-)', 'What(-)', 'Why(-)'] []
	 q: 1 e: 1 ['Who(-)', 'What(-)'] []
	 q: 0 e: 0 ['Who(+)', 'What(+)', 'Who(-